# Preprocessing

## 1. Imports

### 1.1 Libraries

In [1]:
# builtin
import os, time, sys, random
from dotenv import load_dotenv
from pathlib import Path
from datetime import date

# data
import pandas as pd
import numpy as np
import requests
import math

# viz
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# NLP
import string
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.corpus import RegexpTokenizer
from nltk.stem.snowball import FrenchStemmer
from collections import Counter
import torch

# ML
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
import scipy.sparse as sp

# other
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sentence_transformers import SentenceTransformer

### 1.2 Download and options

In [3]:
nltk.download('stopwords')
nltk.download('wordpunct')
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Melvin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading wordpunct: Package 'wordpunct' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Melvin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Melvin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Melvin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
sns.set()

BASE_DIR = Path().resolve().parent
DATA_DIR = BASE_DIR / 'data'

print(f"Répertoire de sauvegarde: {DATA_DIR}\n")

today = date.today()
print("Date du jour :", today)

Répertoire de sauvegarde: C:\Users\Melvin\Desktop\DATA\PORTFOLIO\Recommandation de films\data

Date du jour : 2026-02-10


### 1.3 Loading data

In [5]:
df = pd.read_csv(BASE_DIR / 'data' / 'df_movies_cleaned.csv')

## Variable "Synopsis"

### 2. Work on a specific document

In [6]:
# Fonction de saut de ligne pour output
def insert_newlines(string, every=80):
    lines = []
    for i in range(0, len(string), every):
        lines.append(string[i:i+every])
    return '\n'.join(lines)

In [7]:
doc = df.Synopsis.sample(1)
doc = doc.values[0]
print(insert_newlines(doc, every=200))

Fin des années 70, une équipe de tournage investit une maison isolée du fin fond du Texas pour y réaliser un film X. À la tombée de la nuit, les propriétaires des lieux surprennent les cinéastes amate
urs en plein acte. Le tournage vire brutalement au cauchemar.


### 2.1 Lower

In [8]:
doc = doc.lower()

In [9]:
print(insert_newlines(doc, every=200))

fin des années 70, une équipe de tournage investit une maison isolée du fin fond du texas pour y réaliser un film x. à la tombée de la nuit, les propriétaires des lieux surprennent les cinéastes amate
urs en plein acte. le tournage vire brutalement au cauchemar.


### 2.2 Tokenization

In [10]:
tokens = word_tokenize(doc)
tokens

['fin',
 'des',
 'années',
 '70',
 ',',
 'une',
 'équipe',
 'de',
 'tournage',
 'investit',
 'une',
 'maison',
 'isolée',
 'du',
 'fin',
 'fond',
 'du',
 'texas',
 'pour',
 'y',
 'réaliser',
 'un',
 'film',
 'x.',
 'à',
 'la',
 'tombée',
 'de',
 'la',
 'nuit',
 ',',
 'les',
 'propriétaires',
 'des',
 'lieux',
 'surprennent',
 'les',
 'cinéastes',
 'amateurs',
 'en',
 'plein',
 'acte',
 '.',
 'le',
 'tournage',
 'vire',
 'brutalement',
 'au',
 'cauchemar',
 '.']

In [11]:
# longueur de la liste
len(tokens)

50

In [12]:
# longueur de la liste (sans les doublons)
len(set(tokens))

40

In [13]:
def display_tokens_infos(tokens):
    """display info about corpus"""

    print(f"nb tokens {len(tokens)}, nb tokens uniques {len(set(tokens))}")
    print(tokens[:30])

In [14]:
tokens = wordpunct_tokenize(doc)
display_tokens_infos(tokens)

nb tokens 51, nb tokens uniques 40
['fin', 'des', 'années', '70', ',', 'une', 'équipe', 'de', 'tournage', 'investit', 'une', 'maison', 'isolée', 'du', 'fin', 'fond', 'du', 'texas', 'pour', 'y', 'réaliser', 'un', 'film', 'x', '.', 'à', 'la', 'tombée', 'de', 'la']


### 2.3 Stopwords

In [15]:
stop_words = set(stopwords.words('french'))

In [16]:
tokens = [w for w in tokens if w not in stop_words]
display_tokens_infos(tokens)

nb tokens 32, nb tokens uniques 27
['fin', 'années', '70', ',', 'équipe', 'tournage', 'investit', 'maison', 'isolée', 'fin', 'fond', 'texas', 'réaliser', 'film', 'x', '.', 'tombée', 'nuit', ',', 'propriétaires', 'lieux', 'surprennent', 'cinéastes', 'amateurs', 'plein', 'acte', '.', 'tournage', 'vire', 'brutalement']


In [17]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(doc)
display_tokens_infos(tokens)

nb tokens 46, nb tokens uniques 38
['fin', 'des', 'années', '70', 'une', 'équipe', 'de', 'tournage', 'investit', 'une', 'maison', 'isolée', 'du', 'fin', 'fond', 'du', 'texas', 'pour', 'y', 'réaliser', 'un', 'film', 'x', 'à', 'la', 'tombée', 'de', 'la', 'nuit', 'les']


In [18]:
tokens = [w for w in tokens if w not in stop_words]
display_tokens_infos(tokens)

nb tokens 27, nb tokens uniques 25
['fin', 'années', '70', 'équipe', 'tournage', 'investit', 'maison', 'isolée', 'fin', 'fond', 'texas', 'réaliser', 'film', 'x', 'tombée', 'nuit', 'propriétaires', 'lieux', 'surprennent', 'cinéastes', 'amateurs', 'plein', 'acte', 'tournage', 'vire', 'brutalement', 'cauchemar']


### 2.4 First cleaning function

In [19]:
def process_synopsis_1(doc, rejoin=False):

    # lower
    doc = doc.lower().strip()

    # tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    raw_tokens_list = tokenizer.tokenize(doc)

    # stop words
    cleaned_tokens_list = [w for w in raw_tokens_list if w not in stop_words]

    if rejoin : 
        return " ".join(cleaned_tokens_list)
    
    return cleaned_tokens_list

In [20]:
tokens = process_synopsis_1(doc)
display_tokens_infos(tokens)

nb tokens 27, nb tokens uniques 25
['fin', 'années', '70', 'équipe', 'tournage', 'investit', 'maison', 'isolée', 'fin', 'fond', 'texas', 'réaliser', 'film', 'x', 'tombée', 'nuit', 'propriétaires', 'lieux', 'surprennent', 'cinéastes', 'amateurs', 'plein', 'acte', 'tournage', 'vire', 'brutalement', 'cauchemar']


## 3. Work on the entire corpus

### 3.1 Build raw corpus

In [21]:
raw_corpus = "".join(df.Synopsis.values)
raw_corpus[:3_00]

"Après l'impact dévastateur d'une comète qui a réduit la Terre en ruines, la famille Garrity doit quitter la sécurité de son bunker au Groenland. Commence alors un périple pour leur survie et l'avenir de l'Humanité à travers un monde dévasté à la recherche d'un nouveau foyer.En quête d’un nouveau dép"

In [22]:
len(raw_corpus)

2419538

In [23]:
corpus = process_synopsis_1(raw_corpus)
display_tokens_infos(corpus)

nb tokens 238572, nb tokens uniques 30320
['après', 'impact', 'dévastateur', 'comète', 'a', 'réduit', 'terre', 'ruines', 'famille', 'garrity', 'doit', 'quitter', 'sécurité', 'bunker', 'groenland', 'commence', 'alors', 'périple', 'survie', 'avenir', 'humanité', 'travers', 'monde', 'dévasté', 'recherche', 'nouveau', 'foyer', 'quête', 'nouveau', 'départ']


In [24]:
tmp = pd.Series(corpus).value_counts()
tmp

a             2387
plus          1993
jeune         1379
vie           1304
alors         1116
              ... 
revanchard       1
chul             1
mutilée          1
arènes           1
alexia           1
Name: count, Length: 30320, dtype: int64

In [25]:
tmp.head(10)

a        2387
plus     1993
jeune    1379
vie      1304
alors    1116
deux     1067
tout     1005
va        989
après     850
ans       833
Name: count, dtype: int64

In [26]:
tmp.tail(10)

paillard      1
fisc          1
pitreries     1
pelouses      1
tondues       1
revanchard    1
chul          1
mutilée       1
arènes        1
alexia        1
Name: count, dtype: int64

In [27]:
tmp.describe()

count    30320.000000
mean         7.868470
std         37.864155
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max       2387.000000
Name: count, dtype: float64

### 3.2 List rare tokens

In [28]:
# unique words = usefull ?

tmp = pd.Series(corpus).value_counts()
list_unique_words = tmp[tmp==1]
list_unique_words[:20]

immortalise        1
émue               1
sutra              1
numérisés          1
néfaste            1
collectionne       1
conventionnelle    1
affectera          1
allégorie          1
dépouillé          1
artificial         1
rashida            1
morley             1
fallacieux         1
gorges             1
tangible           1
rocketbelt         1
prostate           1
psychologie        1
prédestinée        1
Name: count, dtype: int64

In [29]:
len(list_unique_words)

13680

In [30]:
list_unique_words = list(list_unique_words.index)
list_unique_words[:20]

['immortalise',
 'émue',
 'sutra',
 'numérisés',
 'néfaste',
 'collectionne',
 'conventionnelle',
 'affectera',
 'allégorie',
 'dépouillé',
 'artificial',
 'rashida',
 'morley',
 'fallacieux',
 'gorges',
 'tangible',
 'rocketbelt',
 'prostate',
 'psychologie',
 'prédestinée']

In [31]:
tmp = pd.DataFrame({"words" : list_unique_words})

save_path = DATA_DIR / 'unique_words.csv'
tmp.to_csv(save_path, index=False, encoding='utf-8')

In [32]:
# idem for min 5 times

tmp = pd.Series(corpus).value_counts()
list_min_5_words = tmp[tmp==5]
list_min_5_words[:20]

mick          5
semblait      5
audace        5
ocean         5
matrice       5
movie         5
étonnantes    5
penchant      5
capturée      5
army          5
député        5
vlad          5
kurt          5
jonny         5
igor          5
associent     5
suédois       5
étendue       5
équipes       5
paisibles     5
Name: count, dtype: int64

In [33]:
len(list_min_5_words)

1123

In [34]:
list_min_5_words = list(list_min_5_words.index)
list_min_5_words[:20]

['mick',
 'semblait',
 'audace',
 'ocean',
 'matrice',
 'movie',
 'étonnantes',
 'penchant',
 'capturée',
 'army',
 'député',
 'vlad',
 'kurt',
 'jonny',
 'igor',
 'associent',
 'suédois',
 'étendue',
 'équipes',
 'paisibles']

In [35]:
tmp = pd.DataFrame({"words" : list_min_5_words})

save_path = DATA_DIR / 'min_5_words.csv'
tmp.to_csv(save_path, index=False, encoding='utf-8')

### 3.3 Second cleaning function

In [36]:
def process_synopsis_2(doc,
                       rejoin=False,
                       list_rare_words=None,
                       min_len_word=2,
                       force_is_alpha="alpha") : 
    
    """cf process_synopsis_1 but with list_unique_words, min_len_word, and force_is_alpha

    positionnal arguments :
    ------------------------
    doc : str : the document (aka a text in str format) to process

    opt args : 
    ------------------------
    rejoin : bool : if True return a string else return the list of tokens
    list_rare_words : list : a list of rare words to exclude
    min_len_word : int : minimum lenght of a word to not exclude
    force_is_alpha : if 1, exclude all tokens with a numeric character

    return : 
    ------------------------
    a string (if rejoin is True) or a list of tokens
    """
    
    # list unique words
    if not list_rare_words:
        list_rare_words = []

    # lower
    doc = doc.lower().strip()

    # tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    raw_tokens_list = tokenizer.tokenize(doc)

    # stop words
    cleaned_tokens_list = [w for w in raw_tokens_list if w not in stop_words]

    #############################################################
    #############################################################

    # no rare tokens
    non_rare_tokens = [w for w in cleaned_tokens_list if w not in list_rare_words]

    # no more len words
    more_than_N = [w for w in non_rare_tokens if len(w) >= min_len_word]

    # only alpha chars
    if force_is_alpha == "alpha":
        alpha_tokens = [w for w in more_than_N if w.isalpha()]
    elif force_is_alpha == "digits4":
        alpha_tokens = [w for w in more_than_N if re.fullmatch(r'\d{4}', w)]
    else:
        alpha_tokens = more_than_N

    #############################################################
    #############################################################

    # manage return type
    if rejoin : 
        return " ".join(alpha_tokens)
    
    return alpha_tokens

In [37]:
display_tokens_infos(corpus)

nb tokens 238572, nb tokens uniques 30320
['après', 'impact', 'dévastateur', 'comète', 'a', 'réduit', 'terre', 'ruines', 'famille', 'garrity', 'doit', 'quitter', 'sécurité', 'bunker', 'groenland', 'commence', 'alors', 'périple', 'survie', 'avenir', 'humanité', 'travers', 'monde', 'dévasté', 'recherche', 'nouveau', 'foyer', 'quête', 'nouveau', 'départ']


In [38]:
len(set(corpus))

30320

In [39]:
corpus = process_synopsis_2(raw_corpus,
                            list_rare_words=list_unique_words,
                            rejoin=False)
display_tokens_infos(corpus)

nb tokens 220573, nb tokens uniques 16401
['après', 'impact', 'dévastateur', 'comète', 'réduit', 'terre', 'ruines', 'famille', 'garrity', 'doit', 'quitter', 'sécurité', 'bunker', 'groenland', 'commence', 'alors', 'périple', 'survie', 'avenir', 'humanité', 'travers', 'monde', 'dévasté', 'recherche', 'nouveau', 'foyer', 'quête', 'nouveau', 'départ', 'millie']


In [40]:
len(set(corpus))

16401

### 3.4 Embeddings

In [41]:
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1018.78it/s, Materializing param=pooler.dense.weight]                              
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [42]:
def process_synopsis_3(doc,
                       rejoin=False,
                       list_rare_words=None,
                       min_len_word=2,
                       force_is_alpha="alpha",
                       use_embeddings=False):
    
    """
    Process synopsis with optional embeddings instead of lemmatization/stemming
    
    Positional arguments:
    ------------------------
    doc : str : the document (aka a text in str format) to process
    
    Optional args:
    ------------------------
    rejoin : bool : if True return a string else return the list of tokens
    list_rare_words : list : a list of rare words to exclude
    min_len_word : int : minimum length of a word to not exclude
    force_is_alpha : if "alpha", exclude all tokens with a numeric character
    use_embeddings : bool : if True, return embeddings vector instead of tokens
    
    Return:
    ------------------------
    - If use_embeddings=True: numpy array (embedding vector)
    - If rejoin=True: string (tokens joined)
    - Otherwise: list of tokens
    """
    
    # List unique words
    if not list_rare_words:
        list_rare_words = []
    
    # Lower
    doc = doc.lower().strip()
    
    # Tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    raw_tokens_list = tokenizer.tokenize(doc)
    
    # Stop words
    cleaned_tokens_list = [w for w in raw_tokens_list if w not in stop_words]
    
    #############################################################
    # Filtering
    #############################################################
    
    # No rare tokens
    non_rare_tokens = [w for w in cleaned_tokens_list if w not in list_rare_words]
    
    # No words shorter than min_len_word
    more_than_N = [w for w in non_rare_tokens if len(w) >= min_len_word]
    
    # Only alpha chars
    if force_is_alpha == "alpha":
        alpha_tokens = [w for w in more_than_N if w.isalpha()]
    elif force_is_alpha == "digits4":
        alpha_tokens = [w for w in more_than_N if re.fullmatch(r'\d{4}', w)]
    else:
        alpha_tokens = more_than_N
    
    #############################################################
    # Embeddings OR tokens
    #############################################################
    
    if use_embeddings:
        # Join tokens into text
        clean_text = ' '.join(alpha_tokens)
        
        # embedding (dense vector capturing meaning)
        embedding = embedding_model.encode(clean_text, convert_to_numpy=True)
        
        return embedding

    #############################################################
    #############################################################

    # Manage return type
    if rejoin:
        return " ".join(alpha_tokens)
    
    return alpha_tokens

In [43]:
from tqdm import tqdm
tqdm.pandas()
print("Création des embeddings pour chaque film...")

df['embeddings'] = df.Synopsis.progress_apply(
    lambda x: process_synopsis_3(x, 
                                 list_rare_words=list_unique_words,
                                 min_len_word=2,
                                 force_is_alpha="alpha",
                                 use_embeddings=True)
)

Création des embeddings pour chaque film...


100%|██████████| 5958/5958 [02:15<00:00, 44.10it/s]


In [44]:
# Créez la matrice d'embeddings
import numpy as np
embeddings_matrix = np.vstack(df['embeddings'].values)

# Calculez la similarité
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(embeddings_matrix, embeddings_matrix)

print("Matrice de similarité créée")

Matrice de similarité créée


In [45]:
print("\n3. FILMS LES PLUS UNIQUES (faible similarité moyenne)")
print("-" * 60)

# Calculez la similarité moyenne de chaque film avec tous les autres
avg_similarities = []
for i in range(len(cosine_sim)):
    # Similarité moyenne (en excluant le film lui-même)
    sim_without_self = np.concatenate([cosine_sim[i][:i], cosine_sim[i][i+1:]])
    avg_similarities.append(sim_without_self.mean())

df['avg_similarity'] = avg_similarities

# Films les plus uniques (faible similarité)
unique_films = df.nsmallest(10, 'avg_similarity')[['Titre', 'avg_similarity']]
print(unique_films.to_string(index=False))

print("\n4. FILMS LES PLUS GÉNÉRIQUES (forte similarité moyenne)")
print("-" * 60)

# Films les plus génériques (forte similarité)
generic_films = df.nlargest(10, 'avg_similarity')[['Titre', 'avg_similarity']]
print(generic_films.to_string(index=False))


3. FILMS LES PLUS UNIQUES (faible similarité moyenne)
------------------------------------------------------------
                             Titre  avg_similarity
                           Airlift        0.061157
                         Bloodwork        0.074734
                             Speed        0.075177
American Strike - L'ultime mission        0.076932
                Popeye le Plombier        0.079250
  American Nightmare 3 : Élections        0.081291
                Situation critique        0.094935
              Le sable était rouge        0.098110
                              Dawn        0.098431
                            Piston        0.099015

4. FILMS LES PLUS GÉNÉRIQUES (forte similarité moyenne)
------------------------------------------------------------
                        Titre  avg_similarity
                Batman Begins        0.424669
                        Cabal        0.414049
            The Family Plan 2        0.406590
Les Cavaliers de l'Ap

In [46]:
def final_clean(doc) :

    new_doc = process_synopsis_3(doc,
                                 rejoin=False,
                                 list_rare_words=list_unique_words,
                                 min_len_word=2,
                                 force_is_alpha="alpha")
    return new_doc

In [60]:
doc

'fin des années 70, une équipe de tournage investit une maison isolée du fin fond du texas pour y réaliser un film x. à la tombée de la nuit, les propriétaires des lieux surprennent les cinéastes amateurs en plein acte. le tournage vire brutalement au cauchemar.'

In [47]:
df['clean_synopsis'] = df.Synopsis.apply(final_clean)

In [48]:
# on reconverti la colonne clean_synopsis_str en chaine de char
df['clean_synopsis_str'] = df['clean_synopsis'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

## 4. Date de sortie to datetime

In [50]:
df['Date de sortie'] = pd.to_datetime(df['Date de sortie'])

### 4.1 Age of movies

In [51]:
current_year = datetime.now().year
df['Age du film'] = current_year - df['Date de sortie'].dt.year

In [52]:
df

,ID,Titre,Date de sortie,Genre,Note,Popularité,Synopsis,Affiche,periode_5_ans,poids_temporel,Note_Catégorie,Pop_Catégorie,Note_Ajustée,Catégorie_Adaptée,embeddings,avg_similarity,clean_synopsis,clean_synopsis_str,Age du film
0,840464,Greenland : Migration,2026-01-07,"['Aventure', 'Thriller', 'Science-Fiction']",6.567,499.8774,Après l'impact dévastateur d'une comète qui a ...,/9eaHOemq2Ch5rYPJ0uXcipZUYpS.jpg,2025,1.000000,Bon (6-7),Très élevée,6.564427,Extrêmement populaire (>18.1),"[-0.027700055, 0.16916879, 0.018964056, -0.224...",0.256877,"[après, impact, dévastateur, comète, réduit, t...",après impact dévastateur comète réduit terre r...,0
1,1368166,La Femme de ménage,2025-12-18,"['Mystère', 'Thriller']",7.178,473.7526,"En quête d’un nouveau départ, Millie accepte u...",/28Dn93nuxH8PmDLUVQLCjLNVxKD.jpg,2025,0.500000,Très bon (7-8),Très élevée,7.167667,Extrêmement populaire (>18.1),"[0.10259485, -0.0042940257, -0.1724262, 0.0959...",0.343441,"[quête, nouveau, départ, millie, accepte, post...",quête nouveau départ millie accepte poste femm...,1
2,1168190,Team Démolition,2026-01-28,"['Action', 'Comédie', 'Crime', 'Mystère']",6.802,418.4280,"Deux demi-frères brouillés, Jonny et James, se...",/ec0bwYUEYBhcHutx4nIShLzX7Dl.jpg,2025,1.000000,Bon (6-7),Très élevée,6.795620,Extrêmement populaire (>18.1),"[0.030473026, 0.04308578, -0.07412312, -0.2527...",0.254323,"[deux, demi, frères, jonny, james, réunissent,...",deux demi frères jonny james réunissent après ...,0
3,1084242,Zootopie 2,2025-11-26,"['Animation', 'Comédie', 'Aventure', 'Familial...",7.600,290.4141,Judy Hopps et Nick Wilde explorent à nouveau Z...,/hBI7Wrps6tDjhEzBxJgoPLhbmT1.jpg,2025,0.500000,Très bon (7-8),Très élevée,7.574757,Extrêmement populaire (>18.1),"[0.119301155, -0.084989265, 0.032965213, 0.155...",0.309305,"[judy, hopps, nick, wilde, explorent, nouveau,...",judy hopps nick wilde explorent nouveau zootop...,1
4,1419406,The Shadow's Edge,2025-08-16,"['Action', 'Crime', 'Drame', 'Thriller']",7.202,262.5255,Un mystérieux mafieux et ses 7 fils adoptifs m...,/t1PFVsGYdUHtPv0Xowoc9b4PAap.jpg,2025,0.500000,Très bon (7-8),Très élevée,7.183004,Extrêmement populaire (>18.1),"[-0.10469549, 0.2563884, -0.25267825, -0.28168...",0.338879,"[mystérieux, mafieux, fils, adoptifs, manipule...",mystérieux mafieux fils adoptifs manipulent po...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5953,341744,Truman,2015-09-24,"['Drame', 'Comédie']",7.019,5.3685,"Julian, un madrilène, reçoit la visite inatten...",/yzoJkuy4XqWx0YfS4WPg3umKKXX.jpg,2015,0.083333,Très bon (7-8),Faible,6.666100,Très peu populaire (<6.0),"[0.08660731, 0.05591605, 0.18804066, -0.209840...",0.247550,"[julian, reçoit, visite, inattendue, ami, toma...",julian reçoit visite inattendue ami tomas vit ...,11
5954,25904,Marketa Lazarova,1967-11-24,"['Drame', 'Histoire']",7.766,5.3683,"Moyen Âge. En Bohême, au XIIIème siècle, chris...",/he6J26FubAydpeBal7NYUe1UeeB.jpg,1965,0.016667,Très bon (7-8),Faible,7.019377,Très peu populaire (<6.0),"[0.030387944, 0.37213928, 0.0035692267, 0.0441...",0.289650,"[moyen, âge, bohême, siècle, christianisme, af...",moyen âge bohême siècle christianisme affronte...,59
5955,2712,Jacob,1994-12-04,"['Aventure', 'Drame', 'Téléfilm']",6.556,5.3677,"Jacob, fils d'Isaac et frère cadet d'Esaü, éco...",/bmwFf3fMgBKqtiST5sQ7wapeLFB.jpg,1990,0.030303,Bon (6-7),Faible,6.447119,Très peu populaire (<6.0),"[0.038031373, 0.40362304, 0.056222953, -0.0559...",0.267756,"[jacob, fils, isaac, frère, cadet, esaü, écout...",jacob fils isaac frère cadet esaü écoute mère ...,32
5956,323214,Alexia,2013-11-04,"['Horreur', 'Thriller']",5.400,5.3676,"Bien qu'Alexia, l'ex-petite amie de Franco, so...",/gJBIFfy4ej4R2ed3OfvGllkg5zw.jpg,2010,0.071429,Moyen (5-6),Faible,5.900443,Très peu populaire (<6.0),"[0.10010207, -0.06224392, 0.072088994, -0.1698...",0.230640,"[bien, ex, petite, amie, franco, décédée, depu...",bien ex petite amie franco décédée depuis cert...,13


## 5. Binary encoding of genres

In [53]:
print(len(df))
print(df['Genre'].apply(lambda x: isinstance(x, str)).sum())

5958
5958


In [54]:
df['Genre'] = df['Genre'].apply(lambda x: eval(x) if isinstance(x, str) else x)
print(df['Genre'].apply(lambda x: isinstance(x, list)).sum())

5958


In [55]:
mlb = MultiLabelBinarizer()
genre_binarized = mlb.fit_transform(df['Genre'])

genre_df = pd.DataFrame(genre_binarized, columns=mlb.classes_)

In [56]:
genre_df.index = df.index

In [57]:
df = pd.concat([df.drop('Genre', axis=1), genre_df], axis=1)

# 6. Final df

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5958 entries, 0 to 5957
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  5958 non-null   int64         
 1   Titre               5958 non-null   object        
 2   Date de sortie      5958 non-null   datetime64[ns]
 3   Note                5958 non-null   float64       
 4   Popularité          5958 non-null   float64       
 5   Synopsis            5958 non-null   object        
 6   Affiche             5958 non-null   object        
 7   periode_5_ans       5958 non-null   int64         
 8   poids_temporel      5958 non-null   float64       
 9   Note_Catégorie      5958 non-null   object        
 10  Pop_Catégorie       5958 non-null   object        
 11  Note_Ajustée        5958 non-null   float64       
 12  Catégorie_Adaptée   5958 non-null   object        
 13  embeddings          5958 non-null   object      

In [59]:
save_path = DATA_DIR / 'df_movies_preprocess.csv'
df.to_csv(save_path, index=False, encoding='utf-8')

save_path = DATA_DIR / 'genres_binarized.csv'
genre_df.to_csv(save_path, index=False, encoding='utf-8')